In [113]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline

import os, pickle
import datetime as dt

# initialize a connection to the database
import sqlite3
conn = sqlite3.connect('../Spotify_Challenge/dbsql.db')

# initialize another sqlalchemy connection to the same database to be able to query data straight to pandas dataframe
from sqlalchemy import create_engine
disk_engine = create_engine('sqlite:///../Spotify_Challenge/dbsql.db')

from IPython.display import HTML

In this notebook we use data collected from [transtats.bts.gov](http://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236&DB_Short_Name=On-Time) to a local database and directly from [faa.gov](http://www.faa.gov/airports/planning_capacity/passenger_allcargo_stats/passenger/previous_years/). We will be making queries to the database using sqlite commands

## Functions

In [151]:
def date_to_week(year,month,day):
    """return week of the year"""
    return dt.date(year,month,day).isocalendar()[1]

## Import data

In [187]:
# get 2014 airports ranking from FAA
links = [[2015,"http://www.faa.gov/airports/planning_capacity/passenger_allcargo_stats/passenger/media/preliminary-cy15-commercial-service-enplanements.xlsx"],
         [2014,"http://www.faa.gov/airports/planning_capacity/passenger_allcargo_stats/passenger/media/cy14-commercial-service-enplanements.xlsx"],
         [2013,"http://www.faa.gov/airports/planning_capacity/passenger_allcargo_stats/passenger/media/cy13-commercial-service-enplanements.xlsx"],
         [2012,"http://www.faa.gov/airports/planning_capacity/passenger_allcargo_stats/passenger/media/CY12CommercialServiceEnplanements.xlsx"],
         [2011,"http://www.faa.gov/airports/planning_capacity/passenger_allcargo_stats/passenger/media/cy11_primary_enplanements.xlsx"],
         [2010,"http://www.faa.gov/airports/planning_capacity/passenger_allcargo_stats/passenger/media/cy10_primary_enplanements.xls"]]
ranks = pd.DataFrame()
for link in sorted(links):
    df = pd.read_excel(link[1])
    df.rename(columns={df.columns[-2]:"Enplanement",df.columns[-1]:"Change"}, inplace=True)
    df['Year'] = link[0]
    ranks = pd.concat([ranks,df[['Year','Rank','Locid','Hub','Enplanement','Change']]],axis=0,ignore_index=True)
    
# only focus on airports classified as large hubs
ranks = ranks[ranks.Hub == "L"]
# drop rows which do not correspond to airports data and reset indices
indices_to_drop = ranks[ranks.Locid.isnull()].index.tolist()
ranks.drop(indices_to_drop,inplace=True)
ranks = ranks.reset_index()
large_hubs = map(lambda x:str(x),pd.unique(ranks[ranks.Hub == "L"]["Locid"]).tolist())

In [211]:
# get aggreagated airports data from the database
airports = pd.read_sql_query("select * from airports_clusters",disk_engine)
# Focus on large hubs only
airports = airports[airports.IATA.isin(large_hubs)].reset_index()[['YEAR','IATA','C1','C2']]
# merge airports with their ranking
airports = pd.merge(airports,ranks[[u'Year', u'Rank', u'Locid', u'Enplanement',u'Change']],
                    how='left',left_on=['YEAR','IATA'],right_on=['Year','Locid'])
airports.drop(['Year','Locid'],axis=1,inplace=True)
airports.columns = map(lambda x:str(x).upper(),airports.columns.tolist())
# get aggregated flights data from the database
flights = pd.read_sql_query("select * from flights_clusters",disk_engine)[['YEAR','ORIGIN','DEST','C']]

In [91]:
features = [u'index',u'YEAR', u'QUARTER', u'MONTH', u'DAY_OF_MONTH', u'DAY_OF_WEEK', u'AIRLINE_ID', 
            u'CARRIER', u'TAIL_NUM', u'FL_NUM', u'ORIGIN_AIRPORT_ID', u'ORIGIN', u'ORIGIN_CITY_NAME', 
            u'ORIGIN_STATE_NM', u'DEST_AIRPORT_ID', u'DEST', u'DEST_CITY_NAME', u'DEST_STATE_NM', 
            u'DEP_DELAY', u'DEP_DELAY_NEW', u'DEP_DEL15', u'TAXI_OUT', u'TAXI_IN', u'ARR_DELAY', 
            u'ARR_DELAY_NEW', u'ARR_DEL15', u'CANCELLED', u'CANCELLATION_CODE', u'DIVERTED', u'AIR_TIME', 
            u'FLIGHTS', u'DISTANCE', u'CARRIER_DELAY', u'WEATHER_DELAY', u'NAS_DELAY', u'SECURITY_DELAY', 
            u'LATE_AIRCRAFT_DELAY',u'FL_DATE']

Let's first define **major airlines** as those airlines that were operational since 2010 until the end of year 2015

In [118]:
tic = dt.datetime.now()
airlines = pd.read_sql_query(
    "SELECT YEAR, CARRIER, COUNT(CARRIER) AS CARRIER_CNT "
    "FROM data "
    "WHERE YEAR <= 2015 "
    "GROUP BY YEAR, CARRIER",
    disk_engine
)
# transform long table to wide
airlines = pd.pivot(airlines.CARRIER,airlines.YEAR,airlines.CARRIER_CNT)

# since we are only interested in airlines that were operational since 2010 until today,
# let's drop all NaN and set columns titles as airlines (of interest)
major_airlines = map(lambda x:str(x),airlines.dropna().index.tolist())
print dt.datetime.now()-tic

0:02:02.476330


In [182]:
# get flights detailed data from the data base
# we are going to only import those features which are known before the flight day: date, origin, destination and airline

tic  =dt.datetime.now()
if False:
    data = pd.read_sql_query("SELECT YEAR, MONTH, DAY_OF_MONTH, DAY_OF_WEEK, ORIGIN, DEST, CARRIER, "
                             "DEP_DEL15 AS DEP_DEL, DEP_DELAY AS DDEL1, DEP_DELAY_NEW AS DDEL2 "
                             "FROM data "
                             "WHERE ORIGIN IN {0} AND DEST IN {0} AND CARRIER IN {1}".format(tuple(large_hubs),
                                                                                             tuple(major_airlines)),
                             disk_engine)
print dt.datetime.now()-tic

0:00:00.000125


In [218]:
tic = dt.datetime.now()
# merge data with airports:
## on origin airport
new_data = pd.merge(data,airports, how='left',left_on=['YEAR','ORIGIN'],right_on=['YEAR','IATA'])
new_data.drop('IATA',axis=1,inplace=True)
## on destination airport
new_data = pd.merge(new_data,airports, how='left',left_on=['YEAR','DEST'],right_on=['YEAR','IATA'])

# merge data with flights
new_data = pd.merge(new_data,flights,how='left', left_on=['YEAR','ORIGIN','DEST'], 
                    right_on=['YEAR','ORIGIN','DEST'])
print dt.datetime.now()-tic

0:01:08.361572


In [219]:
new_data

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,ORIGIN,DEST,CARRIER,DEP_DEL,DDEL1,DDEL2,...,RANK_x,ENPLANEMENT_x,CHANGE_x,IATA,C1_y,C2_y,RANK_y,ENPLANEMENT_y,CHANGE_y,C
0,2010,1,14,4,JFK,BOS,DL,0.0,8.0,8.0,...,6.0,22710272.0,0.0099,BOS,3.0,5.0,19.0,12566797.0,0.0792,18
1,2010,1,14,4,ATL,LAX,DL,0.0,-7.0,0.0,...,1.0,42280868.0,0.0201,LAX,0.0,2.0,3.0,27439897.0,0.0517,14
2,2010,1,14,4,ATL,DTW,DL,0.0,-2.0,0.0,...,1.0,42280868.0,0.0201,DTW,5.0,5.0,15.0,15211402.0,0.0284,18
3,2010,1,14,4,ATL,LAX,DL,0.0,2.0,2.0,...,1.0,42280868.0,0.0201,LAX,0.0,2.0,3.0,27439897.0,0.0517,14
4,2010,1,14,4,ATL,LAX,DL,0.0,-1.0,0.0,...,1.0,42280868.0,0.0201,LAX,0.0,2.0,3.0,27439897.0,0.0517,14
5,2010,1,14,4,LAX,MCO,DL,0.0,-3.0,0.0,...,3.0,27439897.0,0.0517,MCO,5.0,5.0,13.0,16371016.0,0.0395,1
6,2010,1,14,4,JFK,ATL,DL,0.0,5.0,5.0,...,6.0,22710272.0,0.0099,ATL,9.0,5.0,1.0,42280868.0,0.0201,4
7,2010,1,14,4,ATL,MDW,DL,0.0,-4.0,0.0,...,1.0,42280868.0,0.0201,MDW,5.0,1.0,27.0,8253620.0,0.0321,18
8,2010,1,14,4,DTW,ATL,DL,0.0,-6.0,0.0,...,15.0,15211402.0,0.0284,ATL,9.0,5.0,1.0,42280868.0,0.0201,18
9,2010,1,14,4,LAX,ATL,DL,0.0,0.0,0.0,...,3.0,27439897.0,0.0517,ATL,9.0,5.0,1.0,42280868.0,0.0201,14
